За основу взят код с гитхаба
https://github.com/sl-iwi/auto_offers_parser/blob/master/app/auto_ru_parser.py  

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [2]:
from IPython.display import display
pd.options.display.max_columns = None

### Парсинг:

In [3]:
def search_url(vendor):
    #location = 'moskva'
    location = 'rossiya'
    model = 'used'
    url = f"https://auto.ru/{location}/cars/{vendor}/{model}/"
    return url


In [4]:
#Функция парсит данные с нескольких страниц
#%%timeit
# this func returns dataframe with offers.
list_autos = []
def get_autoru_offers_all(u):
#   list_autos = []
#    autos = pd.DataFrame(list_autos)
    
    response = requests.get(u)
    soup = BeautifulSoup(response.content.decode('utf-8'), "html.parser")

    module_list = soup.find("div", class_= "ListingCars-module__container ListingCars-module__list")
    children = module_list.findChildren("div", recursive=False)
    for child in children:
        metas = child.find_all("meta")
        offer = {}
        for meta in metas:
            key = meta.get('itemprop')
            value = meta.get('content')
            offer.update({key: value}) #parse parameters except mileage
        try:        
            miles = child.find("div", {'class': 'ListingItem-module__kmAge'})
            offer.update({'mileage': miles.get_text()}) #parse mileage
        except:
            miles = ''
                    
        list_autos.append(offer)

    next_page = soup.find('link', attrs={'rel': 'next'})
    if next_page is not None:
           # If there is "NEXT" button => continue parsing to the next page
        time.sleep(2)
        next_url = next_page['href']
        get_autoru_offers_all(next_url)
    
    autos = pd.DataFrame(list_autos)
    autos = autos.drop(['image', 'availability', 'priceCurrency'], 1)
    autos.dropna(axis=0, inplace=True) #drop rows with missed values
    autos.reset_index(inplace = True,drop = True)
    return autos

In [5]:
#Функция парсит данные из автомобильного обьявления
#%%timeit
#list_car = pd.DataFrame()
def get_autoru_car_info(adr):
    car_info = pd.DataFrame(columns = ['Привод', 'Руль', 'Состояние', 'Владельцы', 'ПТС', 'Таможня'])
#    list_car = pd.DataFrame()
    list_car = []
    response = requests.get(adr)
    soup = BeautifulSoup(response.content.decode('utf-8'), "html.parser")
    mod_list = soup.find("ul", class_="CardInfo")
    offer = {}
    offer.update({'url': adr})
    row_drive = mod_list.find("li", class_="CardInfo__row CardInfo__row_drive").text[6:]
    offer.update({'Привод': row_drive})
    row_wheel = mod_list.find("li", class_="CardInfo__row CardInfo__row_wheel").text[4:]
    offer.update({'Руль': row_wheel})
    row_state = mod_list.find("li", class_="CardInfo__row CardInfo__row_state").text[9:]
    offer.update({'Состояние': row_state})
    row_ownersCount = mod_list.find("li", class_="CardInfo__row CardInfo__row_ownersCount").text[9:]
    offer.update({'Владельцы': row_ownersCount})  
    row_pts = mod_list.find("li", class_="CardInfo__row CardInfo__row_pts").text[3:]
    offer.update({'ПТС': row_pts})  
    row_customs = mod_list.find("li", class_="CardInfo__row CardInfo__row_customs").text[7:]
    offer.update({'Таможня': row_customs}) 
    try:
        row_owningTime = mod_list.find("li", class_="CardInfo__row CardInfo__row_owningTime").text[8:]
    except:
        row_owningTime = ''
    offer.update({'Владение': row_owningTime})
    card_description = soup.find("div", class_="CardDescription__textInner").text
    offer.update({'description': card_description})
   
    # парсинг блока с комплектацией
    cr = soup.find(string = re.compile("equipmentGroups"))
    index_start = cr.find('"equipmentGroups"') + 18
    index_end = cr.find('"owner_expenses"') - 2
    #print(cr[index_start:index_end])
    
    #в переменной names хранятся все ключи блока комплектации
    #names = soup.find_all("div", class_="CardComplectation-module__CardComplectation__itemName")

    offer.update({'Комплектация': cr[index_start:index_end]})
    
    list_car.append(offer)
        
    car_info = pd.DataFrame(list_car)
    
    return car_info

In [6]:
#функция принимает на вход марку авто и далее парсит сначала перечень обьявлений из поисковой выдачи, а затем для каждого обьявления по url парсит данные из конкретного обьявления
def parsing_func(brand):
    
    u = search_url(brand)
    
    train_set = get_autoru_offers_all(u)

    print(train_set.shape)

    counter = train_set['url'].shape[0]
    inside_list=[]
    for i in range(0, counter-1):
        try:
            inside_list.append(get_autoru_car_info(train_set['url'][i]))
        except:
            train_set['url'][i] = ''
    additional_data = pd.concat(inside_list, axis = 0, join = 'outer', ignore_index = False)
    train_set = train_set.merge(additional_data, how='left')
    print(train_set.shape)
    return train_set

In [7]:
#пример парсинга данных по марке VW с дальнейшей записью в файл
list_autos = []
train_vw = parsing_func('volkswagen')
train_vw.to_csv('train_vw_27march.csv')

(3663, 15)
(3753, 24)


In [21]:
train_vw = pd.read_csv('train_vw_27march.csv')
train_audi = pd.read_csv('train_audi_27march.csv')
train_bmw = pd.read_csv('train_bmw_27march.csv')
train_toyota = pd.read_csv('train_toyota_27march.csv')
train_merc = pd.read_csv('train_mercedes_27march.csv')
train_skoda = pd.read_csv('train_skoda_27march.csv')
train_nissan = pd.read_csv('train_nissan_27march.csv')
train_mit = pd.read_csv('train_mit_27march.csv')
train_volvo = pd.read_csv('train_volvo_27march.csv')
train_honda = pd.read_csv('train_honda_27march.csv')
train_lexus = pd.read_csv('train_lexus_27march.csv')
train_inf = pd.read_csv('train_inf_27march.csv')
train_suzuki = pd.read_csv('train_suzuki_27march.csv')

In [22]:
df_r = pd.concat([train_vw, train_audi, train_bmw,train_toyota,train_merc,train_skoda, train_nissan, train_mit, train_volvo, train_honda, train_lexus, train_inf, train_suzuki], axis = 0, join = 'outer', ignore_index = False)

In [23]:
df_r.shape

(48454, 25)

In [24]:
cols = ['bodyType', 'brand', 'color', 'fuelType', 'modelDate', 'name',
       'numberOfDoors', 'productionDate', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement',
       'enginePower','description', 'mileage', 'Комплектация','Привод', 'Руль', 'Состояние', 'Владельцы',
       'ПТС', 'Таможня', 'Владение', 'price', 'url']

In [25]:
df_r = df_r[cols]

In [27]:
df_r.sample(5)

,bodyType,brand,color,fuelType,modelDate,name,numberOfDoors,productionDate,vehicleConfiguration,vehicleTransmission,engineDisplacement,enginePower,description,mileage,Комплектация,Привод,Руль,Состояние,Владельцы,ПТС,Таможня,Владение,price,url
2664,внедорожник 5 дв.,NISSAN,чёрный,бензин,2010,2.5 CVT,5,2011,ALLROAD_5_DOORS VARIATOR 2.5,вариатор,2.5 LTR,169 N12,ОДИН ВЛАДЕЛЕЦ !!!!!!!!!!.............ПТС ОРИГИ...,157 000 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",полный,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,NaN,729000,https://auto.ru/cars/used/sale/nissan/x_trail/...
3384,седан,AUDI,красный,бензин,1990,2.8 MT,4,1991,SEDAN MECHANICAL 2.8,механическая,2.8 LTR,174 N12,Продаю отличного и надёжного немца в спортивно...,280 000 км,"[{""name"":""Комфорт"",""values"":[""Электростеклопод...",передний,Левый,Не требует ремонта,3 или более,Дубликат,Растаможен,NaN,200000,https://auto.ru/cars/used/sale/audi/100/109585...
3491,седан,BMW,чёрный,дизель,2016,2.0 AT,4,2019,SEDAN AUTOMATIC 2.0,автоматическая,2.0 LTR,190 N12,Без дтп в отличном состоянии.,30 000 км,"[{""name"":""Салон"",""values"":[""Тонированные стекл...",задний,Левый,Не требует ремонта,1 владелец,Оригинал,Растаможен,7 месяцев,2300000,https://auto.ru/cars/used/sale/bmw/5er/1094800...
2672,лифтбек,AUDI,белый,бензин,2007,2.0 AMT,5,2010,LIFTBACK ROBOT 2.0,роботизированная,2.0 LTR,211 N12,"Продаю любимую машину, чистокровная немка приг...",211 000 км,"[{""name"":""Салон"",""values"":[""Тонированные стекл...",полный,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,4 года и 7 месяцев,850000,https://auto.ru/cars/used/sale/audi/a5/1095880...
1354,внедорожник 5 дв.,VOLKSWAGEN,чёрный,бензин,2011,1.4 AMT,5,2013,ALLROAD_5_DOORS ROBOT 1.4,роботизированная,1.4 LTR,150 N12,Данный автомобиль представлен в автосалоне Мег...,93 000 км,"[{""name"":""Комфорт"",""values"":[""Круиз-контроль"",...",передний,Левый,Не требует ремонта,2 владельца,Оригинал,Растаможен,NaN,625000,https://auto.ru/cars/used/sale/volkswagen/tigu...


In [28]:
df_r_f = df_r.drop_duplicates(keep='first', inplace=False)
df_r_f.shape

(46998, 24)

In [29]:
df_r_f.to_csv('train_data_27march.csv')